In [17]:
from langchain_community.llms import Ollama

# model
llm = Ollama(model='llama3')

# chain execute

llm.invoke("지구의 자전 주기는?, 한국어로 번역해줘")

'😊\n\nThe length of a day on Earth is approximately 24 hours. 🕰️\n\nIn Korean:\n\n 지구의 자전 주기는 약 24시간입니다.\n\n(Note: "자전" means "rotation" or "revolution", and "주기" means "period" or "cycle".)'

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

output_parser = StrOutputParser()

In [19]:
chain = prompt | llm | output_parser

In [20]:
print(chain.invoke({'input': "how can langsmith help with testing?"}))

As a world-class technical documentation writer, I'm excited to share how Langsmith can assist with testing.

Langsmith's AI-powered writing capabilities can significantly enhance the testing process by generating test cases, test data, and even testing scripts. Here are some ways Langsmith can help:

1. **Test Case Generation**: Langsmith can analyze requirements and generate test cases based on the functionality, ensuring comprehensive coverage.
2. **Test Data Creation**: The AI can create realistic test data for different scenarios, reducing the need to manually generate data and increasing test efficiency.
3. **Testing Scripts**: Langsmith can assist in generating testing scripts, including step-by-step instructions for manual or automated testing.
4. **Defect Reporting**: When defects are found during testing, Langsmith can help generate clear, concise reports outlining the issue, steps to reproduce it, and expected results.
5. **Test Summary Reports**: The AI can summarize test r

In [32]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://ttoro.tistory.com/89")

docs = loader.load()

In [33]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [34]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [35]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""주어진 내용을 오직 기반으로 질문에 대답해줘:
                                          
                                          <context>
                                          {context}
                                          <context>
                                          
                                          Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [37]:
from langchain_core.documents import Document

resp = document_chain.invoke({
    "input": "벽지하자 문제를 어떻게 접근했나요?",
    "context": [Document(page_content="classification model인 CNN과 yolov5를 통해 문제를 해결했습니다.")]
})

print(resp)

Based on the context, it seems that the problem is related to image classification or object detection.

Answer: We approached the problem using a classification model (CNN) and YOLOv5. 🤔


In [38]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [39]:
response = retrieval_chain.invoke({"input": "벽지하자 문제를 어떻게 접근했나요?"})
print(response['answer'])

#

According to the given content, the approach taken for the wall crack problem is as follows:

* Initially, a classification-based approach was attempted, using various pre-trained CNN models (EfficientNet, ResNet, VGG) and image augmentation techniques to prevent overfitting.
* However, due to the high degree of variation in the images and the presence of noise and gaps in the wall cracks, the accuracy remained stuck at around 50%.
* To overcome this challenge, a segmentation-based approach was adopted, focusing on detecting specific features within the images rather than classifying the entire image.
* The team used Yolo models (yolo_v5s, v5m, v5l, v8s, v8m, v8l) and trained them using labeled data from approximately 3,000 images, with each team member manually labeling around 600 images.
* Although the accuracy was not high, the model was able to detect some of the wall cracks reasonably well.
* The team recognized that this approach had limitations and that a more accurate solution 

In [40]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [50]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="벽지하자 분석을 어떻게 했는지 알겠어?"), AIMessage(content="네!")]
result = retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "어떻게 했는지 알려줘"
})

In [55]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [56]:
chat_history = [HumanMessage(content="벽지하자분석을 어떻게 했는지 알겠어?"), AIMessage(content="Yes!")]
result = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [60]:
for key, value in result.items():

    if key == 'context':
        pass
    else:
        print(f'{key} : {value}')

chat_history : [HumanMessage(content='벽지하자분석을 어떻게 했는지 알겠어?'), AIMessage(content='Yes!')]
input : Tell me how
answer : Based on the context, I'll answer your questions.

**Q1: What was the goal of the AIVLE 3rd project?**
A1: The goal was to develop an AI model that can accurately classify the types of defects in images based on a dataset provided by DACON (Dacon's Official Description).

**Q2: How did you approach this problem?**
A2: You approached it as a classification problem first, using various pre-trained CNN models and image augmentation techniques. However, you found that this approach didn't improve the score much beyond 50%. Therefore, you shifted to a segmentation approach, focusing on detecting defects in images.

**Q3: What were some of the challenges you faced during the project?**
A3: One major challenge was the difficulty in labeling the data due to the variability in image features. You also faced issues with overfitting and underfitting, as well as the need for more p